# Цели и задачи

**Цель** этого ipynb-файла и соответствующего Python отображения – генерировать html для соответствующей страницы [matyushkin.github.io/posts](https://matyushkin.github.io/posts).

Данные о статьях берутся со страницы [github.com/matyushkin/lessons/](https://github.com/matyushkin/lessons/) и csv-файла `posts.csv`. Файл содержит:
* `title` — заголовок статьи
* `url` — полная ссылка на публикацию
* `type` — тип публикации (перевод, статья, инструкция, курс или подборка)
* `date` — дата публикации
* `views` — число просмотров
* `main` — главный тег статьи
* `addtional` — дополнительные теги
* `raiting` — мой личный рейтинг отношения к статье
* `comment` — краткий комментарий

# Текущие задачи

* Проверяем и дополняем файл на месте: если ссылка в таблице есть, то обновляем информацию по README-файлу
* Привести ссылки в датафрейме и в readme к единообразному виду -- слеш на конце
* пока не рассматривались задачи - надо добавить в обход для функции парсера (все то же самое)
* сравнить ссылки в файле ../lessons/README.md и в файле. Если каких-то ссылок нет, то описать их в файле
* число просмотров можно отображать в виде серого верхнего индекса в конце названия
* Отдельно определять число просмотров теста.
* Обрабатывать ошибки Connection Error -- делать пропуск и последующее обращение в конце списка до тех пор, пока список не будет целиком обновлён или ошибка не будет повторяться более 3 раз (статья удалена или что-то подобное).
* Привести к виду отдельного самостоятельно выполняющегося Python-файла, который будет выводить лишь системную информацию.

Соберем данные из файла

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'

df = pd.read_csv("posts.csv")

# преобразуем url к идентичному виду относительно слэша на конце url
df['url'] = df['url'].apply(lambda url: url[:-1] if url[-1] == '/' else url)

In [ ]:
df

# Сбор данных из README-файла репозитория `lessons`

Прежде, чем начать парсинг страниц, соберем данные из локальной версии репозитория [lessons](https://github.com/matyushkin/lessons). В этот README-файл я добавляю ссылки в первую очередь. Если соответствующих ссылок нет в таблице статей, их туда нужно добавить.

In [ ]:
import re

def readme_parser(df, path = '../../lessons/README.md'):
    data = {}
    with open(path) as readme_file:
        f = readme_file.read()
        lines = re.findall(r'(?<=- ).*(?=\n)', f)
        exers = re.findall(r'(?<=\n)\d{1,2}\. .*', f)
        for line in lines:
            t = re.findall('(?<=\[).*?(?=\])', line)
            u = re.findall('(?<=\()http.*?(?=\))', line)
            j = re.findall('(?<=\[Jupyter\]\()http\S*ipynb', line)
            c = re.findall('(?<=\[Colab\]\()http.*ipynb', line)
            s = re.findall('(?<=\[ист.\]\()http.*(?<!\))', line)
            
            title = t[0] if t else ''
            url = u[0] if u else ''
            url = url[:-1] if (not url or url[-1] == '/') else url
            jupyter_url = j[0] if j else ''
            colab_url = c[0] if c else ''
            source_url = s[0] if s else ''
            
            if line.find('[') > 0:
                emoji = line[:line.find('[')].strip()
            else:
                emoji = ''
            if not (df['url'].str.contains(url).any()):
                result = {'title': title,
                          'url': url,
                          'jupyter': jupyter_url,
                          'colab': colab_url,
                          'source': source_url,
                          'emoji': emoji}
                for key in result:
                    print(key, result[key])
                df = df.append(result, ignore_index=True)
            else:
                df.loc[df['url'] == url, 'jupyter'] = jupyter_url
                df.loc[df['url'] == url, 'colab'] = colab_url
                df.loc[df['url'] == url, 'source'] = source_url
                df.loc[df['url'] == url, 'emoji'] = emoji
    return df

df = readme_parser(df)

In [ ]:
df

# Просмотры

Данные о просмотрах обновляются, в особенности для новых статей, которые я писал для [Библиотеки программиста](https://proglib.io). Пропарсим страницы новых публикаций и обновим сведения о просмотрах.

In [ ]:
df_proglib_articles = df[df['url'].apply(lambda x: 'https://proglib.io/p/' in x)]
df_proglib_tests = df[df['url'].apply(lambda x: 'https://proglib.io/tests/' in x)]
df_habr_articles = df[df['url'].apply(lambda x: 'https://habr.com/' in x)]


def proglib_article_get_views_number(url:str):
    '''Возвращает число просмотров для переданного url'''
    page = requests.get(url, headers=headers).text
    soup = BeautifulSoup(page, 'html.parser')
    span = soup.body.find_all('span', {"class":"ml-1", "data-views":""})[0]
    return int(span.text)


def proglib_test_get_views_number(url:str):
    '''Возвращает число просмотров для переданного url'''
    page = requests.get(url, headers=headers).text
    soup = BeautifulSoup(page, 'html.parser')
    span = soup.body.find_all('div', {"class": "reactions-bar__views"})[0]
    return int(span.text.strip())


def habr_get_views_number(url:str):
    '''Возвращает число просмотров для переданного url'''
    page = requests.get(url, headers=headers).text
    soup = BeautifulSoup(page, 'html.parser')
    span = soup.body.find_all('span', {"class":"post-stats__views-count"})[0]
    result = span.text.replace(',', '.')
    if 'k' in result:
        result = float(result[:-1])*1000
    return int(result)


def get_views(df_part, type_of_page):
    for url in df_part['url']:
        print(url)
        if type_of_page == 'proglib_article':
            num = proglib_article_get_views_number(url)
        elif type_of_page == 'proglib_test':
            num = proglib_test_get_views_number(url)
        elif type_of_page == 'habr_article':
            num = habr_get_views_number(url)
        print(num)
        df.loc[df['url'] == url, 'views'] = num

get_views(df_habr_articles, 'habr_article')
get_views(df_proglib_articles, 'proglib_article')
get_views(df_proglib_tests, 'proglib_test')

In [ ]:
df

In [ ]:
df['views'].sum()

In [ ]:
df.to_csv('posts.csv', index=False)

# Создание index.html из шаблона (template.html)

In [ ]:
# Добавить обработку комментария, проверку на nan

import collections

def gen_h(soup, title, level=1):
    h_tag = soup.new_tag('h'+str(level))
    h_tag.string = title
    return h_tag
  

def gen_li(soup, title, url, date, views,
           rating, typ):
    li_tag = soup.new_tag('li')
    li_tag["data-views"] = views
    li_tag["data-rating"] = rating
    li_tag["data-type"] = typ
    a_tag = soup.new_tag('a', href=url)
    time_tag = soup.new_tag('time', datetime=date)
    time_tag.string = title
    a_tag.insert(0, time_tag)
    li_tag.insert(0, a_tag)
    return li_tag


def gen_ul(soup, selected:dict):
    ul_tag = soup.new_tag('ul')
    ul_tag.append('\n')
    for key in selected["title"].keys():
        title = selected["title"][key]
        url = selected["url"][key]
        date = selected["date"][key]
        views = selected["views"][key]
        rating = selected["rating"][key]
        typ = selected["type"][key]
        comment = selected["comment"][key]
        ul_tag.append(' '*4)
        ul_tag.append(gen_li(soup, title, url, date, views,
                             rating, typ))
        ul_tag.append('\n')
    ul_tag.append(' '*2)
    return ul_tag


with open("../../mgio/11ty/_includes/postcards.njk") as template:
    soup = BeautifulSoup(template, "lxml")
    article_tag = soup.find('article')
    cf = df.copy()
    c = collections.Counter(df['main'])
    for (title, num) in c.most_common():
        if num >= 5:
            article_tag.append(gen_h(soup, title))
            article_tag.append('\n')
            selected = cf[cf['main']==title].sort_values(by=['date'],
                                                         ascending=False).to_dict()
            cf = cf[cf['main']!=title]
            article_tag.append(' '*2)
            article_tag.append(gen_ul(soup, selected))
            article_tag.append('\n')
    article_tag.append(gen_h(soup, "Публикации по другим темам"))
    article_tag.append('\n')
    selected = cf.sort_values(by=['date'], ascending=False).to_dict()
    article_tag.append(' '*2)
    article_tag.append(gen_ul(soup, selected))
    article_tag.append('\n')
        
    html_file = open("index.html", "w")
    html_file.write(soup.prettify())
    html_file.close()